In [3]:
import os
from pathlib import Path
import pandas as pd


In [4]:
# set paths
DATAPATH = "./data"
Path(DATAPATH).mkdir(parents=True, exist_ok=True)

print('The working directory is: ', os.getcwd())

The working directory is:  h:\Andere Computer\Mein Computer\GoogleDrive\Beruf\Freelancing\Code_Repo\Customer_Analytics\CreditDefaultPrediction


# Import Dataset

In [10]:
df = pd.read_csv(DATAPATH + '/loan_default.csv')
df.head()

In [12]:
# show data dimensions
df.shape

(148670, 34)

In [13]:
# show data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10